In [1]:
from helipad import Helipad
from helipad.utility import CobbDouglas
from math import sqrt, exp, floor
import random

heli = Helipad()
heli.name = "Price Discovery"

Define the order othat the agents will be called in

options

* random: call agents randomly

* linear: call agents in order they were instantiated

* match: call agents randomly and pair randomly

* order() allows for explicitly defined order of agents

* stages can allow for lists of agents to be called multiple times within a period (step)

In [2]:
heli.order  = "match"

Helipad.addParameter( str name, str title, str type, str|bool|num|list dflt, dict|list opts = {}, bool runtime = True, func callback = None, str desc = None)

1. Set exogenous parameters.

2. Name will be used to refer to the parameter. It is the name of the parameter.

3. Title will be the title of the slider (or other widget) in the cpanel

4. "slider"  chooses the type of widget.
    Parameters could be hidden.

5. _dflt_ sets the initial value of the parameter.

6. _opts_ sets the bounds and step of the slider. 

In [3]:
heli.addParameter("ratio", "Log Endowment Ratio", "slider", 
                  dflt = 0, opts = {"low": -3, "high":3, "step":.5})

In [4]:
#heli.launchCpanel()

Everything above Log Endowment Ratio is the default template for Cpanel. You can see that Log Endowment Ratio is a slider with range that we specified.

Next, create 

In [5]:
# keyword to change to num instead agents
# next, target values on the default slider for Number of Agents
# makes sure that slider moves in incriments of 2
heli.params["agents_agent"].opts["step"] = 2
# and make sure that the slider's lowest value is 2 
heli.params["agents_agent"].opts["low"] = 2

Now the Cpanel will reflect the change in the Number of Agents slider. Scroll the slider to see that moves in incriments of 2 and has a minimum value of 2.

In [6]:
#heli.launchCpanel()

Next we will define the goods that each agent owns and the distributions that determine agent allocation.

In [7]:
heli.addParameter("ratio", "Log Endowment Ratio", "slider", 
                  dflt = 0, opts = {"low": -3, "high":3, "step":.5})

# keyword to change to num instead agents
# next, target values on the default slider for Number of Agents
# makes sure that slider moves in incriments of 2
heli.params["agents_agent"].opts["step"] = 2
# and make sure that the slider's lowest value is 2 
heli.params["agents_agent"].opts["low"] = 2

max_shmoo = 1000
# addGood(name, color, endowment)
heli.addGood("shmoo", "11CC00", lambda breed: random.randint(1,max_shmoo))

# floor rounds value down
# exp(param) raises e to the param power 
heli.addGood("soma", "CC0000", 
            endowment =  lambda breed: random.randint(
                1, floor(exp(heli.param("ratio")) * max_shmoo)))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Parameter 'ratio' already defined. Overriding…
  


So far, no agents have been created. We will continue defining agent attributes. Next, define agent utility functions using @heli.hook. @heli is a decorator. A decorator is a function that passes a function to another function.

We are using hook to define attributes and/or methods that are automatically called when agents are instantiated.

call hook method and pass agentInit() to the hook method.

Hook is a method of heli that allows you to 

Before we call agent Init, it is essentially blank. This is how give agents special characteristics aside from their endowment.

In [8]:
# agentInit() already exists, we are further defining for our model
@heli.hook
def agentInit(agent, model):
    agent.utility = CobbDouglas(["shmoo", "soma"])

In [9]:
@heli.hook
def match(agents, primitive, model, stage):
    # agent 0 and agent 1 calculate their utility
    u1e = agents[0].utility.calculate(agents[0].stocks)
    u2e = agents[1].utility.calculate(agents[1].stocks)
    
    # Get the endpoints of the contract curve
    # Contract curve isn't linear unless the CD function
    cc1Soma = u1e * (sum([a.stocks['soma'] for a in agents])/sum(
        [a.stocks['shmoo'] for a in agents])) ** 0.5
    # note: draw exponent from model instead of mannually choosing it
    cc2Soma = sum([a.stocks['soma'] for a in agents]) - u2e  * (
        sum([a.stocks['soma'] for a in agents])/sum(
            [a.stocks['shmoo'] for a in agents])) ** 0.5
    cc1Shmoo = sum([a.stocks['shmoo'] for a in agents])/sum(
        [a.stocks['soma'] for a in agents]) * cc1Soma
    cc2Shmoo = sum([a.stocks['shmoo'] for a in agents])/sum(
        [a.stocks['soma'] for a in agents]) * cc2Soma
    
    # Calculate demand: choopse a random point on the contract curve
    	#Do the trades
    
    min_trade_quantity = 0.1
    if abs(somaDemand) > min_trade_quantity and abs(shmooDemand) > min_trade_quantity:
        # agent.trade(partner, payment_good, q_payment, receive_good, q_receive) 
        agents[0].trade(agents[1], 'soma', -somaDemand, 'shmoo', shmooDemand)
        agents[0].lastPrice = -somaDemand/shmooDemand
        agents[1].lastPrice = -somaDemand/shmooDemand
    else:
        agents[0].lastPrice = None
        agents[1].lastPrice = None

    # Calculate new utility levels
    agents[0].utils = agents[0].utility.calculate(
        agents[0].stocks)
    agents[1].utils = agents[1].utility.calculate(
        agents[1].stocks)

Next, collect data

In [10]:
#heli.data.addReporter(name, reporter_function,
heli.data.addReporter('ssprice', 
  heli.data.agentReporter('lastPrice', 'agent', stat='gmean',
      percentiles=[0,100]))

# stat option: 'mean', 'sum', 'gmean' (for geometric mean), 
# 'std' (for standard deviation), or 'percentile-nn'

Next let's plot the data

In [11]:
# addPlot(name, title, logscale, selected)
heli.addPlot("price", "Price", logscale = True, selected = True)
heli.addSeries("price", "ssprice", "Soma/Shmoo Price", "119900")

In [12]:
#Stop the model when we're basically equilibrated
def stopCondition(model):
	return model.t > 1 and model.data.getLast('demand-shmoo') < 20 and model.data.getLast('demand-soma') < 20
heli.param('stopafter', stopCondition)

for p in ['demand', 'utility']: heli.plots[p].active(True)
heli.param('updateEvery', 1)


In [13]:
heli.launchCpanel()

HTML(value='<style type="text/css">/* The color indicator on per-item parameters */\n\n.helipad_circle {\n\the…

interactive(children=(IntSlider(value=50, description='Number of Agents', description_tooltip='', min=2, step=…

interactive(children=(FloatSlider(value=0.0, description='Log Endowment Ratio', description_tooltip='', max=3.…

Accordion(children=(HBox(children=(interactive(children=(Checkbox(value=True, description='Demand', descriptio…

Label(value='After setting parameter values, run launchPlots() or start() to start the model.', _dom_classes=(…

In [14]:

heli.launchPlots()

Task exception was never retrieved
future: <Task finished coro=<Helipad.run() done, defined at C:\ProgramData\Anaconda3\lib\site-packages\helipad\model.py:462> exception=NameError("name 'somaDemand' is not defined")>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\helipad\model.py", line 465, in run
    t = self.step()
  File "C:\ProgramData\Anaconda3\lib\site-packages\helipad\model.py", line 447, in step
    self.doHooks([prim+'Match', 'match'], [agents, prim, self, self.stage])
  File "C:\ProgramData\Anaconda3\lib\site-packages\helipad\model.py", line 377, in doHooks
    r = self.doHooks(f, args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\helipad\model.py", line 382, in doHooks
    for f in self.hooks[place]: r = f(*args)
  File "<ipython-input-9-b23196b18e6f>", line 24, in match
    if abs(somaDemand) > min_trade_quantity and abs(shmooDemand) > min_trade_quantity:
NameError: name 'somaDemand' is not defined
